# Comparison of Old AuxTel ISR with New ISR with parallel overscan and crosstalk subtraction - crosstalk transposed
Craig Lage 09-Jan-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.summit.utils import BestEffortIsr
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    vmin = 1
    vmax = 100
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return


In [ ]:
butler_before = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])
butler_after = Butler('/repo/embargo', collections=["LATISS/raw/all", "u/cslage/sdf/calib/latiss/calib_par_cross_20230106"])
best_effort_isr_before = BestEffortIsr(embargo=True)

In [ ]:
expId = 2022112300453

 ### Old ISR

In [ ]:
dataId = {'exposure': expId, 'detector': 0, "instrument": 'LATISS'}
best_effort_isr_before.doWrite = False  # Don't write to butler database
exp_before = best_effort_isr_before.getExposure(dataId, skipCosmics=False)
fig = plotExp(exp_before)
plt.suptitle(f"{expId}")
fig.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Old_ISR_Stretch_{expId}_09Jan23.png")

 ### New ISR

In [ ]:
crosstalkValues = [0.0, 0.00024470391931360916, 4.883246913991542e-05, 2.0418182505347234e-05, 
8.898295927180605e-06, 6.773341098404962e-06, 3.570388309862894e-06, 5.3809067647701876e-06, 
-1.4481694074673388e-06, -9.689480922044601e-07, -1.485865544154329e-06, -9.637919133282712e-07, 
-1.2157572968266757e-06, -5.217045366898451e-07, 4.044232855652071e-06, 2.0352791713644413e-05, 
0.00015888336550941553, 0.0, 0.00022323979903339305, 7.663526881495104e-05, 
4.65761873524069e-05, 2.6008291467554955e-05, 2.8678016067781073e-06, -5.974419921699883e-06, 
-2.2408697277009e-06, 8.974784665907462e-07, -4.539848436053557e-06, -1.8983453205184659e-06, 
-3.452216875169628e-06, 4.530352324585823e-06, 3.996308730923587e-06, 2.7116153215408716e-05, 
3.788170293684901e-05, 0.00024947603368651955, 0.0, 0.0003415121003992023, 
8.73116649498964e-05, 4.5520412670111093e-05, 1.9224563170350935e-06, -1.1662353446167973e-05, 
-2.467604183126412e-07, 2.2849177906583825e-06, -3.1595824899728166e-06, 5.822135176750107e-07, 
8.576571717394367e-06, 1.0403948446637116e-05, 1.1157114544501083e-05, 6.723175399578788e-06, 
9.933872708581732e-06, 9.297956862262597e-05, 0.0002487835770449991, 0.0, 
0.00016188949003024533, 5.742005252408831e-05, -5.1836956685920735e-06, -2.496420912380099e-05, 
-1.3027392735589607e-06, -1.6389002289195602e-06, -2.6332349303015907e-06, -2.1457459716798389e-07, 
6.931280141888806e-06, 1.483849579390179e-05, -1.8884775083145607e-06, -2.0175516412651795e-06, 
2.808792195062614e-06, 5.446469509101461e-05, 9.894601615698697e-05, 0.00015514286041579597, 
0.0, 0.0008733650985158913, 2.5277367322461668e-05, -1.6052633113115148e-05, 
7.122879763875737e-07, 3.3461772927228364e-06, 3.904609186469354e-05, 6.645508457169517e-06, 
6.894078160263231e-06, -9.62056945142165e-07, -2.025439123703041e-06, -2.5835984896554303e-06, 
1.8421191510313597e-06, 3.538220478337849e-05, 5.366857753726588e-05, 7.041020945318627e-05, 
0.000315733058600579, 0.0, 0.00023742201438061643, -1.2147390477178905e-05, 
-7.007800224518524e-08, 3.951058635509022e-06, 1.1884998685517373e-05, 1.926819670425349e-05, 
-1.4980521409811762e-06, -4.93685489256739e-07, -3.012761343863761e-06, -1.274883122174834e-06, 
4.966259648498857e-06, 1.1957245107398185e-05, 9.079460052123976e-06, 1.2284805090899454e-05, 
2.752723616423848e-05, 0.00015300159309239293, 0.0, 0.0015736490591040963, 
1.879343859736736e-06, 2.4221686306780196e-05, 2.274159647392664e-05, 7.447047745231304e-07, 
5.992244803228838e-07, -1.665593245841829e-06, -1.0480865551281963e-06, 7.844131996879105e-07, 
6.220491564786351e-06, 5.655566532679767e-06, 8.505620991978652e-07, 4.2558818752883366e-06, 
1.4940550528526992e-06, 1.6055211172715017e-05, 0.00020504699706798372, 0.0, 
1.678337466644244e-05, 2.0546367186506874e-05, 5.7685644342233356e-08, 3.825948955469393e-07, 
3.081419545762707e-07, -2.174795767942349e-06, 2.74988973524529e-06, 2.4959646616587955e-06, 
1.229941066093052e-06, -2.193297731923316e-06, -5.84796012740521e-07, 4.103817209748911e-06, 
4.746205564044835e-06, -1.21810326108837e-06, 2.653159842143788e-06, 1.6185385800755344e-05, 
0.0, 0.0004224723146914864, 4.394301200448987e-06, -8.015332919326301e-06, 
-9.666880102236e-06, -3.5696336874929715e-06, -1.4791305908240787e-07, 2.6614996964618528e-06, 
-4.290528236003061e-06, 4.151001613108685e-07, 4.2899918266257187e-07, -2.1305952486134125e-06, 
-1.9923891449017904e-06, 2.3454208759065637e-06, 1.6515581022895524e-05, 1.865945124130999e-05, 
0.00022000465144973125, 0.0, 0.00011332098015803947, -1.2268754097035193e-05, 
-3.989115414104398e-06, -5.808772657213313e-06, -5.559135420195798e-06, -4.100325037808096e-06, 
-1.3358423925506902e-06, -9.528997434546441e-07, -5.724454719871431e-07, 2.078402993151939e-09, 
3.640889101000278e-05, 1.1334640536096042e-05, 1.9002148038475157e-05, 5.545970397133101e-07, 
-2.7977204287922815e-06, 0.00020216398757165246, 0.0, 0.0017424803242983159, 
5.02346924795469e-05, 3.4896997694271714e-05, 1.4615136582101152e-05, -5.9425651905724865e-06, 
5.228198537834383e-07, 1.1898088833525855e-06, 8.062468235047843e-07, -7.787395422421039e-07, 
4.179757960643622e-06, 1.6892974681087563e-05, 2.2004012962711287e-06, 8.376130053936916e-07, 
-1.500465665417047e-05, 3.22067824426738e-05, 0.0005444913758968536, 0.0, 
0.00011909236372512782, 6.91895795472634e-05, 2.9672866035619164e-05, -1.5249106914395756e-05, 
-4.892069111840228e-07, 5.003457515107364e-07, 1.0728940459583974e-05, 3.0633993912083894e-06, 
7.757089759628362e-06, -1.3913890742945033e-06, -1.452248744973186e-06, -7.349465296681093e-07, 
-1.1519801156143158e-05, 1.7164475567926765e-05, 6.90614024902783e-05, 0.00016250543823652442, 
0.0, 0.0003851940438104246, 6.1608581592689e-05, -7.608841653502559e-06, 
-1.5012114269596314e-06, 4.893182430672108e-07, 7.719473939032656e-06, 1.223631733619101e-05, 
-1.0496853914689421e-06, -1.2765033732767498e-06, -2.178756429296533e-06, -2.5632316944152853e-06, 
-7.484669107701178e-06, 1.4505332705203746e-05, 5.156488718053364e-05, 9.385502798784448e-05, 
0.00025176337260515475, 0.0, 0.00017482083464693908, -8.733207451592442e-06, 
3.188762205226405e-06, 2.4353974480931923e-06, 9.665914087644667e-06, -1.343852891018992e-07, 
-1.2559215898123022e-06, -1.8268192780564383e-06, -3.332491895848686e-06, -2.021606318243888e-06, 
2.1149656673713017e-06, 1.3119574333202956e-05, 3.519552059843461e-05, 5.7752745638708104e-05, 
8.982143769863497e-05, 0.00021422904840776453, 0.0, 0.0007388418987397647, 
1.9728818979472125e-05, 2.420465431645097e-05, 5.883924026251512e-06, 2.7237079592636606e-06, 
-1.5341247851894657e-06, 5.051324335010207e-07, -4.599476434974772e-07, -1.2077754576503793e-06, 
1.0986495368415558e-05, 1.415051316742601e-05, 1.3090319993608439e-05, 1.5943970053133675e-05, 
1.8028988544747377e-05, 3.1458432126703984e-05, 0.00026596894698489897, 0.0]

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=True
isrConfig.crosstalk.useConfigCoefficients=True
isrConfig.crosstalk.crosstalkShape=16,16
isrConfig.crosstalk.crosstalkValues=crosstalkValues
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
exp = butler_after.get('raw', detector=0, exposure=expId)
biasExp = butler_after.get('bias', detector=0, exposure=expId)
#flatExp = butler_after.get('flat', detector=0, exposure=expId)
darkExp = butler_after.get('dark', detector=0, exposure=expId)
defectExp = butler_after.get('defects', detector=0, exposure=expId)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
isrResult = isrTask.run(exp, bias=biasExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/New_ISR_Stretch_Trans_{expId}_09Jan23.png")

In [ ]:
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/New_ISR_Stretch_{expId}_09Jan23.png")